In [0]:
# Install all necessary packages in one cell
%pip install -U mlflow databricks-openai databricks-agents tabulate

# Restart Python to ensure all new packages are available
dbutils.library.restartPython()

In [0]:
def nimble_health_providers(city: str, display: bool = False):
    df = spark.sql(f"SELECT * FROM `dais-hackathon-2025`.nimble.dbx_google_maps_search_daily where array_contains(business_category, 'Health')")
    pdf = df.toPandas()
    pdf = pdf[pdf.city == city]
    # stringify arrays and complex data
    for col in pdf.columns:
        if pdf[col].apply(lambda x: isinstance(x, (list, tuple, np.ndarray))).any():
            pdf[col] = pdf[col].apply(str)
    # display
    if display:
        df.show()
    # return
    return pdf

#Consolidated code:

In [0]:

%%writefile quickstart_agent.py
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

import mlflow
import json
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient

import uuid
from typing import Any, Optional


import pandas as pd
import numpy as np


# MODEL_NAME = "databricks-claude-sonnet-4"
# MODEL_NAME = "databricks-claude-3-5-sonnet",
MODEL_NAME = "databricks-meta-llama-3-1-8b-instruct"

# Get an OpenAI client configured to connect to Databricks model serving endpoints
# Use this client to query the LLM
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

# Enable automatic tracing for easier debugging
mlflow.openai.autolog()

# Load Databricks built-in tools (Python code interpreter)
client = DatabricksFunctionClient()

class ToolResult:
  def __init__(self, value):
    self.value = value

tools = []

# weather

def get_weather(city: str) -> ToolResult:
  if city == "Tokyo":
    value = "sunny"
  elif city == "Paris":
    value = "rainy"
  else:
    value = "unknown"
  return ToolResult(value)
weather_tool = {
  "type": "function",
  "function": {
    "name": "get_weather",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {"type": "string"}
      }
    }
  }
}
tools.append(weather_tool)

# health data
def get_health_provider_data(city: str) -> ToolResult:
  pdf = nimble_health_providers(city, True)
  for col in pdf.columns:
      if pdf[col].apply(lambda x: isinstance(x, (list, tuple, np.ndarray))).any():
          pdf[col] = pdf[col].apply(str)
  pdf_markdown_str = pdf.to_markdown(index=False)
  return ToolResult(pdf_markdown_str)
#
get_health_provider_data_tool = {
  "type": "function",
  "function": {
    "name": "get_health_provider_data",
    "parameters": {
      "city": {
        "type": "string",
        "description": "The city for which to retrieve health provider data."
      }
    },
  }
}
tools.append(get_health_provider_data_tool)


def call_tool(tool_name, parameters):
  if tool_name == "get_weather":
    return get_weather(**parameters)
  elif tool_name == "get_health_provider_data":
    return get_health_provider_data(**parameters)
  raise ValueError(f"Unknown tool: {tool_name}")

# ...
def addToolResultToPrompt(prompt, tool_name, tool_result_value):
  return (
    f"{prompt}\n\n"
    f"NOTE: The tool '{tool_name}' was run and returned the following result: {tool_result_value}\n"
    "Please use this result to answer the user's question."
  )

# ...
def run_agent(prompt):
  """
  Send a user prompt to the LLM and return a list of LLM response messages
  The LLM is allowed to call the code interpreter tool, if needed, to respond to the user
  """

  system_prompt = (
    "You are an assistant that can use external tools. "
    "If a tool result is available, always use the tool result to answer the user's question. "
    "If no tool result is available, proceed as usual."
  )

  toolResultPresent = False
  while True:
    result_msgs = []
    msgs = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": prompt},
    ]
    # only use tools on the first call
    if toolResultPresent:
      insertTools = []
    else:
      insertTools = tools
    # call
    response = openai_client.chat.completions.create(
      model=MODEL_NAME,
      messages=msgs,
      tools=insertTools,
    )
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())
    # If the model executed a tool, get the result
    if msg.tool_calls:
      call = msg.tool_calls[0]
      tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
      prompt = addToolResultToPrompt(prompt, call.function.name, tool_result.value)
      toolResultPresent = True
      continue
    # return after getting a result from the LLM assistant
    break
  return result_msgs


# comment for testing
class QuickstartAgent(ChatAgent):
  def predict(
    self,
    messages: list[ChatAgentMessage],
    context: Optional[ChatContext] = None,
    custom_inputs: Optional[dict[str, Any]] = None,
  ) -> ChatAgentResponse:
    # 1. Extract the last user prompt from the input messages
    prompt = messages[-1].content

    # 2. Call run_agent to get back a list of response messages
    raw_msgs = run_agent(prompt)

    # 3. Map each response message into a ChatAgentMessage and return
    # the response
    out = []
    for m in raw_msgs:
      out.append(ChatAgentMessage(
        id=uuid.uuid4().hex,
        **m
      ))

    return ChatAgentResponse(messages=out)

AGENT = QuickstartAgent()
mlflow.models.set_model(AGENT)


#Log agent:

In [0]:
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution

MODEL_NAME = "databricks-meta-llama-3-1-8b-instruct"
CATALOG_NAME = "team12a"

# Change the catalog name ("main") and schema name ("default") to register the agent to a different location
registered_model_name = f"{CATALOG_NAME}.default.quickstart_agent"

# Specify Databricks resources that the agent needs to access.
# This step lets Databricks automatically configure authentication
# so the agent can access these resources when it's deployed.
resources = [
  DatabricksServingEndpoint(endpoint_name=MODEL_NAME),
]

mlflow.set_registry_uri("databricks-uc")
  
logged_agent_info = mlflow.pyfunc.log_model(
  artifact_path="agent",
  python_model="quickstart_agent.py",
  extra_pip_requirements=[f"databricks-connect=={get_distribution('databricks-connect').version}"],
  resources=resources,
  registered_model_name=registered_model_name
)

mlflow.end_run()

#Deploy:

In [0]:
from databricks import agents

deployment_info = agents.deploy(
  model_name=registered_model_name, model_version=logged_agent_info.registered_model_version
)